In [2]:
import os
import json
from zipfile import ZipFile

import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
kaggle_dictionary = json.load(open('kaggle.json'))

In [4]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [5]:
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [6]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [8]:
data = pd.read_csv('IMDB Dataset.csv')
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [9]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [10]:
dataset = data.copy()

In [11]:
# Replacing words with 1 and 0 s
dataset.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)
dataset

C:\Users\HP\AppData\Local\Temp\ipykernel_10320\2912495443.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [12]:
dataset['sentiment'].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

Preparing Data for Preprocessing

In [13]:
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

In [14]:
train_data.shape, test_data.shape

((40000, 2), (10000, 2))

Data Preprocessing

In [15]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen= 200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen= 200)

In [16]:
print(X_train)
print(X_test)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]
[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [17]:
y_train = train_data['sentiment']
y_test = test_data['sentiment']

LSTM (Long Short-Term Memory) Model

In [18]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))  # First LSTM layer
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))  # Second LSTM layer
model.add(Dropout(0.5))  # Dropout layer with 50% dropout rate
model.add(Dense(64, activation='relu')) 
model.add(Dense(1, activation='sigmoid'))

c:\Users\HP\Desktop\Data Analytics + Science\Deep Learning Projects\Sentiment Analysis on IMDB Reviews with LSTM DPL\.myvenv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [19]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
# Training the Model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 384s 749ms/step - accuracy: 0.7034 - loss: 0.5458 - val_accuracy: 0.8564 - val_loss: 0.3517
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 386s 771ms/step - accuracy: 0.8521 - loss: 0.3630 - val_accuracy: 0.8066 - val_loss: 0.4203
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 297s 593ms/step - accuracy: 0.8666 - loss: 0.3332 - val_accuracy: 0.8367 - val_loss: 0.3869
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 282s 565ms/step - accuracy: 0.8728 - loss: 0.3155 - val_accuracy: 0.8673 - val_loss: 0.3116
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 301s 602ms/step - accuracy: 0.8996 - loss: 0.2596 - val_accuracy: 0.8781 - val_loss: 0.2948
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 272s 544ms/step - accuracy: 0.9126 - loss: 0.2229 - val_accuracy: 0.8867 - val_loss: 0.2757
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 279s 558ms/step - accuracy: 0.9294 - loss: 0.1949 - val_accuracy: 0.8901 - val_loss: 0.2818
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 279s 558ms/step - accuracy: 0.9372 -

In [21]:
loss, accuracy = model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 36s 114ms/step - accuracy: 0.8825 - loss: 0.3239


Building a Prediction System

In [22]:
def predict_sentiment(review):
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = 'positive' if prediction[0][0] > 0.5 else 'negative'
    return sentiment

In [23]:
input_review = "This Movie has a crazy plot. I do not like it"
sentiment = predict_sentiment(input_review)
print(f'The Sentiment of the Review is: {sentiment}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 750ms/step
The Sentiment of the Review is: positive


In [24]:
model.save('Movie_Reviews_Model.h5')

In [25]:
with open('tokenizer.json', 'w') as t:
    t.write(tokenizer.to_json())